## This is very important!
You need to generate the dataset yourself! You can do dynamic generation on the go, but it usually takes around 1.5 hours to go through the dataset. I prefer to use statically generated dataset, but I don't think that it is worth sharing. Here is the code to generate it:

In [2]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import json
import pickle

cuda = torch.device('cpu')
frame_size = 10
ratings = pd.read_csv('../data/ml-20m/ratings_lite.csv')
movies = pickle.load(open('../data/infos_pca128.pytorch', 'rb'))
infos_web = json.load(open('../data/infos.json')) 

# credits: KnightofK9
ratings["rating"] = ratings["rating"].apply(lambda i: 2 * (i - 2.5))
users = ratings[["userId","movieId"]].groupby(["userId"]).size()
users = users[users >= frame_size + 1]

In [3]:
test_users = users[-100:]
train_users = users[:-100]

train_ratings = ratings[ratings["userId"].isin(train_users.index)]
test_ratings = ratings[ratings["userId"].isin(test_users.index)]

train_ratings = train_ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")
test_ratings = test_ratings.sort_values(by=["userId", "timestamp"]).drop(columns=["timestamp"]).set_index("userId")

for i in movies.keys():
    movies[i] = movies[i].to(cuda)

In [4]:
import io
import matplotlib.pyplot as plt
import pickle

n_iter = 1
n_batch = 1
n_user = 0


def get_minibatch(df, idx):
    user_ratings = df[idx:frame_size + idx + 1]
    user_ratings = user_ratings[["movieId", "rating"]].values

    chosen_movie = user_ratings[:, 0][-1] 
    chosen_movie = movies[chosen_movie] # action
    chosen_rating = user_ratings[:, 1][-1] # reward
    films_watched = user_ratings[:, 0][:-1]
    watched_rating = user_ratings[:, 1][:-1] # state
    watched_infos = [movies[i] for i in films_watched] # state
    watched_infos = torch.stack(watched_infos)
    next_infos = torch.cat((watched_infos[1:], chosen_movie.unsqueeze(0)), 0)
    next_rating = watched_rating[1:].tolist()
    next_rating.append(chosen_rating)
    next_rating = np.array(next_rating)

    # state action reward next_state done
    return [(watched_infos, watched_rating), chosen_movie, chosen_rating,
                  (next_infos, next_rating), idx + 1 == size]


batch_bar = tqdm(total=len(train_users))
batch = []
batch_size = 100

for user, df in train_ratings.groupby(level=0):
    batch_bar.update(1)
    n_user += 1
    size = max(len(df) - frame_size, 0)
    for idx in range(0, size):
        if np.random.rand() < 0.8:  # intake percents
            continue
        batch.append(get_minibatch(df, idx))
    
    # if the memory error pops up, change the number below
    # 8gb ram = 5000
    # 16gb ram = 10000
    # 32gb ram = 30000
    if n_user % 10000 == 0:
        pickle.dump(batch, open('../data/batches/batch_{}.p'.format(n_batch), 'wb'))
        n_batch += 1
        del batch
        batch = []


KeyboardInterrupt: 

In [ ]:
pickle.dump(batch, open('../data/batches/batch_{}.p'.format(n_batch), 'wb'))
n_batch += 1
del batch
batch = []